In [20]:
# def Starting_Lineup(date):
import pybaseball as pyb
import pandas as pd
import time
import inspect
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import optim
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import os
from featureranker.utils import view_data
from featureranker.rankers import classification_ranking
from featureranker.rankers import voting
import glob
import numpy as np
from tqdm.auto import tqdm
import pickle
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup
pyb.cache.enable()
pyb.cache.config.cache_type='csv'
pyb.cache.config.save()

# Functions

# Importing Data

# Formatting player data

# Formatting per game data

In [21]:
from bs4 import BeautifulSoup
import re
import pybaseball as pyb
date = '2022-07-23'
# Fetch the HTML content from the webpage
url = f'https://www.mlb.com/starting-lineups/{date}'
response = requests.get(url)
html_content = response.text

# Assuming `html_content` contains the HTML source you provided
from bs4 import BeautifulSoup
import re

# Assuming `html_content` contains the HTML source you provided
from bs4 import BeautifulSoup
import re
import pybaseball as pyb

# Assuming `html_content` contains the HTML source you provided
soup = BeautifulSoup(html_content, 'html.parser')

# Regular expression to extract numeric ID from player URL
player_id_pattern = re.compile(r'/player/.*-(\d+)$')

# Collect all MLBAM IDs
all_mlbam_ids = set()

# Find all game containers
games = soup.find_all('div', class_='starting-lineups__matchup')

for game in games:
    # Extract pitcher and batter IDs
    pitchers = game.find_all('a', class_='starting-lineups__pitcher--link')
    batters = game.find_all('a', class_='starting-lineups__player--link')
    player_links = pitchers + batters
    
    for player_link in player_links:
        player_id_match = re.search(player_id_pattern, player_link['href'])
        if player_id_match:
            all_mlbam_ids.add(int(player_id_match.group(1)))

# Perform reverse lookup to get Retro IDs
all_player_info = pyb.playerid_reverse_lookup(list(all_mlbam_ids), key_type='mlbam')
mlbam_to_retro = all_player_info.set_index('key_mlbam')['key_retro'].to_dict()


game_matchups = {}

# Use enumerate to get both the index and the game object for uniqueness
for index, game in enumerate(games):
    teams = game.find_all('span', class_='starting-lineups__team-name')
    away_team_code = teams[0].find('a')['data-tri-code']
    home_team_code = teams[2].find('a')['data-tri-code']
    
    # Extract pitcher IDs
    pitchers = game.find_all('a', class_='starting-lineups__pitcher--link')
    if len(pitchers) < 3:  # Ensure there are at least 3 links, indicating both away and home pitchers are present
        continue  # Skip this game if there aren't enough pitcher links
    away_pitcher_id = re.search(player_id_pattern, pitchers[0]['href']).group(1) if len(pitchers) > 0 else None
    home_pitcher_id = re.search(player_id_pattern, pitchers[2]['href']).group(1) if len(pitchers) > 1 else None
    if not away_pitcher_id or not home_pitcher_id:
        continue
    # Extract batters IDs for both teams
    away_batters = game.find('ol', class_='starting-lineups__team--away').find_all('a', class_='starting-lineups__player--link')
    home_batters = game.find('ol', class_='starting-lineups__team--home').find_all('a', class_='starting-lineups__player--link')
    
    away_batters_ids = [re.search(player_id_pattern, batter['href']).group(1) for batter in away_batters]
    home_batters_ids = [re.search(player_id_pattern, batter['href']).group(1) for batter in home_batters]
    if not away_batters_ids or not home_batters_ids:
        continue
    # Populate the game_matchups dictionary with a unique game key
    game_key = f"{away_team_code}@{home_team_code}_{index}"
    game_matchups[game_key] = {
        'away_team': away_team_code,
        'home_team': home_team_code,
        'away_pitcher': {'mlbam_id': away_pitcher_id, 'retro_id': mlbam_to_retro.get(int(away_pitcher_id)) if away_pitcher_id else None},
        'home_pitcher': {'mlbam_id': home_pitcher_id, 'retro_id': mlbam_to_retro.get(int(home_pitcher_id)) if home_pitcher_id else None},
        'away_players': [{'mlbam_id': id, 'retro_id': mlbam_to_retro.get(int(id))} for id in away_batters_ids],
        'home_players': [{'mlbam_id': id, 'retro_id': mlbam_to_retro.get(int(id))} for id in home_batters_ids]
    }

In [22]:
# print(game_matchups)

{'CLE@CWS_0': {'away_team': 'CLE', 'home_team': 'CWS', 'away_pitcher': {'mlbam_id': '663474', 'retro_id': 'mcket001'}, 'home_pitcher': {'mlbam_id': '456501', 'retro_id': 'cuetj001'}, 'away_players': [{'mlbam_id': '680757', 'retro_id': 'kwans001'}, {'mlbam_id': '642708', 'retro_id': 'rosaa003'}, {'mlbam_id': '608070', 'retro_id': 'ramij003'}, {'mlbam_id': '614177', 'retro_id': 'reyef001'}, {'mlbam_id': '665926', 'retro_id': 'gimea001'}, {'mlbam_id': '680911', 'retro_id': 'millo002'}, {'mlbam_id': '666134', 'retro_id': 'jonen002'}, {'mlbam_id': '595978', 'retro_id': 'hedga001'}, {'mlbam_id': '664702', 'retro_id': 'stram002'}], 'home_players': [{'mlbam_id': '543281', 'retro_id': 'harrj002'}, {'mlbam_id': '660162', 'retro_id': 'moncy001'}, {'mlbam_id': '683734', 'retro_id': 'vauga001'}, {'mlbam_id': '547989', 'retro_id': 'abrej003'}, {'mlbam_id': '518735', 'retro_id': 'grany001'}, {'mlbam_id': '657757', 'retro_id': 'sheeg001'}, {'mlbam_id': '664874', 'retro_id': 'zavas001'}, {'mlbam_id': '

# Set-up Neural Network

# Predict the outcome of the season and calculate profit

In [23]:
# Define a mapping of incorrect team abbreviations to correct ones
team_nomenclature_mapping = {
    'CHC': 'CHN',  # Chicago Cubs
    'WSH': 'WAS',  # Washington Nationals
    'NYM': 'NYN',  # New York Mets
    'BOS': 'BOS',  # Boston Red Sox
    'ATL': 'ATL',  # Atlanta Braves
    'TEX': 'TEX',  # Texas Rangers
    'HOU': 'HOU',  # Houston Astros
    'KC ': 'KCA',   # Kansas City Royals
    'LAA': 'ANA',  # Los Angeles Angels
    'MIN': 'MIN',  # Minnesota Twins
    'MIL': 'MIL',  # Milwaukee Brewers
    'AZ ': 'ARI',   # Arizona Diamondbacks
    'SD ': 'SDN',   # San Diego Padres
    'TB ': 'TBA',   # Tampa Bay Rays
    'OAK': 'OAK',  # Oakland Athletics
    'LAD': 'LAN',  # Los Angeles Dodgers
    'SF ': 'SFN',   # San Francisco Giants
    'SEA': 'SEA',  # Seattle Mariners
    'CWS': 'CHA',  # Chicago White Sox
    'CLE': 'CLE',  # Cleveland Guardians
    'DET': 'DET',  # Detroit Tigers
    'PHI': 'PHI',  # Philadelphia Phillies
    'PIT': 'PIT',  # Pittsburgh Pirates
    'STL': 'SLN',  # St. Louis Cardinals
    'TOR': 'TOR',  # Toronto Blue Jays
    'BAL': 'BAL',  # Baltimore Orioles
    'MIA': 'MIA',  # Miami Marlins
    'CIN': 'CIN',  # Cincinnati Reds
    'COL': 'COL',  # Colorado Rockies
    'NYY': 'NYA',  # New York Yankees
    'AZ': 'ARI',   # Arizona Diamondbacks
    'SD': 'SDN',   # San Diego Padres
    'TB': 'TBA',   # Tampa Bay Rays
    'SF': 'SFN',   # San Francisco Giants
    'KC': 'KCA',   # Kansas City Royals
    # ... add other mappings as needed
}
# ... (previous code)

# Create a list to hold all game data
games_data = []

# Create a dictionary to keep track of game counts for the day
game_counts = {}

# Iterate over the game matchups to populate the games_data list
for game, details in game_matchups.items():
    # Correct the team abbreviations
    home_team_corrected = team_nomenclature_mapping.get(details['home_team'], details['home_team'])
    away_team_corrected = team_nomenclature_mapping.get(details['away_team'], details['away_team'])

    # Create a unique game identifier based on the corrected teams and the number of games they've played that day
    game_count_key = f"{home_team_corrected}{away_team_corrected}"
    game_counts[game_count_key] = game_counts.get(game_count_key, 0) + 1
    game_id = f"{date.replace('-', '')}{home_team_corrected}{away_team_corrected}{str(game_counts[game_count_key])}"  # Ensure the game number is two digits

    # Define game_data inside the loop to reset it for each game
    game_data = {
        'Game_ID': game_id,
        'HmStPchID': details['home_pitcher'].get('retro_id') if isinstance(details['home_pitcher'], dict) else None,
        'VisStPchID': details['away_pitcher'].get('retro_id') if isinstance(details['away_pitcher'], dict) else None
    }
    
    # Add Home team batters in the order they were scraped
    for i, player in enumerate(details['home_players'], start=1):
        game_data[f'HmBat{i}ID'] = player.get('retro_id') if isinstance(player, dict) else None
    
    # Add Away team batters in the order they were scraped
    for i, player in enumerate(details['away_players'], start=1):
        game_data[f'VisBat{i}ID'] = player.get('retro_id') if isinstance(player, dict) else None
    
    # Append the game_data dictionary to the games_data list once per game
    games_data.append(game_data)

# ... (rest of your code to convert games_data to DataFrame and export)

# Convert the games_data list to a DataFrame
games_df = pd.DataFrame(games_data)

# Define the column order explicitly to match the desired structure
column_order = ['Game_ID', 'HmStPchID', 'VisStPchID'] + \
               [f'HmBat{i}ID' for i in range(1, 10)] + \
               [f'VisBat{i}ID' for i in range(1, 10)]

# Reorder the DataFrame columns
games_df = games_df[column_order]

# Export the DataFrame to an Excel file
games_df.to_excel('pybaseball_v3_saved/Scraping_Games.xlsx', index=False)
# return games_df

In [24]:
print(games_df)

            Game_ID HmStPchID VisStPchID  HmBat1ID  HmBat2ID  HmBat3ID  \
0   20220723CHACLE1  cuetj001   mcket001  harrj002  moncy001  vauga001   
1   20220723SEAHOU1  gilbl002   verlj001  crawj002  frant002  winkj002   
2   20220723BOSTOR1  crawk001   manoa001  duraj001  vazqc001  verda001   
3   20220723PHICHN1  wheez001   strom001  schwk001  hoskr001  castn001   
4   20220723DETMIN1  pinem001   ryanj002  greer003  baezj001  grosr001   
5   20220723CINSLN1  minom001   matzs001  indij001  drurb001  phamt001   
6   20220723PITMIA1  quinj001   meyem001  newmk001  alleg002  hayek001   
7   20220723BALNYA1  lylej001   coleg001  mullc002  rutsa001  manct001   
8   20220723NYNSDN1  bassc001   snelb001  nimmb001  marts002  lindf001   
9   20220723KCATBA1  singb001   patil001  melem001  wittb002  benia002   
10  20220723MILCOL1  woodb005   urenj001  yelic001  adamw002  tellr001   
11  20220723CHACLE2  lynnl001   pilkk001  andet001  moncy001  vauga001   
12  20220723LANSFN1  uriaj001   wooda0